In [1]:
import pandas as pd
import numpy as np
import os
from itertools import combinations, product

from PopSynthesis.Methods.connect_HH_PP.paras_dir import processed_data, geo_lev, output_dir
from PopSynthesis.Generator_data.generate_combine_census.utils import TRS
from PopSynthesis.Methods.connect_HH_PP.scripts.const import *
from PopSynthesis.Methods.connect_HH_PP.scripts.sample_hh_main import *
from PopSynthesis.Methods.connect_HH_PP.scripts.sample_pp import *
from PopSynthesis.Methods.connect_HH_PP.scripts.process_all_hh_pp import *

In [2]:
i = 0
# import the del hh
del_hh = pd.read_csv(os.path.join(processed_data, "keep_check", f"del2_df_hh_{i}.csv"))
# import the kept hh
kept_hh = pd.read_csv(os.path.join(processed_data, "keep_check", f"hh2_keep_df_{i}.csv"))
# import original
ori_census = pd.read_csv(os.path.join(data_dir, "hh_marginals_ipu.csv"), header=[0,1])
# test how to

C:\Users\dlaa0001\AppData\Local\Temp\ipykernel_17224\282176772.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  kept_hh = pd.read_csv(os.path.join(processed_data, "keep_check", f"hh2_keep_df_{i}.csv"))


In [3]:
cols = [x for x in kept_hh.columns if x not in ALL_RELA and x not in ["POA", "hhid"]]
ls_temp_hold = []
for att in cols:
    kept_hh[att] = kept_hh[att].astype(str)
    temp_hold = kept_hh.groupby(geo_lev)[att].value_counts().unstack().fillna(0)
    temp_hold.columns = [(temp_hold.columns.name, x) for x in temp_hold.columns]
    temp_hold = temp_hold.astype(int)
    ls_temp_hold.append(temp_hold)
marg_new_raw = pd.concat(ls_temp_hold, axis=1)
convert_marg_dict = {com_col: marg_new_raw[com_col] for com_col in marg_new_raw.columns}
convert_marg_dict[("zone_id", None)] = marg_new_raw.index
new_marg_hh = pd.DataFrame(convert_marg_dict)
ls_drop_m = list(new_marg_hh.columns[new_marg_hh.columns.get_level_values(0).isin(["zone_id"])])
new_marg_hh = new_marg_hh.drop(columns=ls_drop_m)
new_marg_hh

dwelltype                                                 \
     Flat or Apartment Missing Other Separate House Terrace/Townhouse   
POA                                                                     
3000             21326      91     0              4                 0   
3002              1824       5    49             29               457   
3003              3109       8    30             29               635   
3004              5570      16     0             77                 0   
3006             11608      26    93             76                 0   
...                ...     ...   ...            ...               ...   
3990                11       0     4             20                26   
3991                 0       0     9            175                 1   
3992                 0       0     0            528                 0   
3995                 1       4   254           4475               114   
3996                 0       2     0           2464               238   

            owndwell                                         hhinc  ...  \
     Being Purchased Being Rented Fully Owned Something Else 1-149  ...   
POA                                                                 ...   
3000            3137        13061        4871            352   405  ...   
3002             455         1270         609             30    40  ...   
3003             669         2115         983             44    45  ...   
3004             816         3362        1381            104    71  ...   
3006            1750         7146        2721            186   163  ...   
...              ...          ...         ...            ...   ...  ...   
3990              18           24          19              0     2  ...   
3991              59           35          90              1     5  ...   
3992             162           94         265              7    10  ...   
3995            1417          812        2551             68    65  ...   
3996             783          454        1426             41    34  ...   

     totalvehs      hhsize                                   
             3   4+      1     2     3    4    5   6  7  8+  
POA                                                          
3000      1029  355  10379  8509  1815  598   98  11  2   9  
3002       116   58   1034  1015   178   86   34   5  0  12  
3003       241   86   1504  1586   457  209   37   8  6   4  
3004       317   99   2564  2372   477  213   24   5  4   4  
3006       645  195   5109  5010  1231  382   48   8  7   8  
...        ...  ...    ...   ...   ...  ...  ...  .. ..  ..  
3990        18    9      6    20     8    4    2   0  0  21  
3991        34   26     37    65    15   19   10   9  0  30  
3992        88   64    134   200    70   58   24   9  0  33  
3995       571  322   1663  1834   584  443  221  67  7  29  
3996       284  120    805  1176   231  292  134  30  5  31  

[691 rows x 43 columns]

In [4]:
ls_drop = list(ori_census.columns[ori_census.columns.get_level_values(0).isin(["zone_id", "sample_geog"])])
new_ori_census = ori_census.set_index(ori_census[ori_census.columns[ori_census.columns.get_level_values(0)=="zone_id"]].iloc[:, 0])
new_ori_census = new_ori_census.drop(columns=ls_drop)
new_ori_census

hhsize                                   \
                                   1     2     3    4    5   6  7  8+   
(zone_id, Unnamed: 0_level_1)                                           
3000                           10393  8524  2026  656  118  27  4  58   
3002                            1035  1019   190  101   38   9  0  55   
3003                            1504  1591   476  223   40  11  7  55   
3004                            2566  2382   511  227   33   6  5  55   
3006                            5113  5033  1280  414   59  13  8  58   
...                              ...   ...   ...  ...  ...  .. ..  ..   
3990                               6    20     8    4    3   0  0  55   
3991                              37    65    16   19   10   9  0  55   
3992                             134   201    72   60   24   9  0  55   
3995                            1663  1834   629  451  228  73  8  58   
3996                             806  1176   251  299  137  31  6  59   

                                      dwelltype          ...      hhinc  \
                              Flat or Apartment Missing  ... Nil income   
(zone_id, Unnamed: 0_level_1)                            ...              
3000                                      21545     130  ...       1898   
3002                                       1852      10  ...         42   
3003                                       3165       5  ...        158   
3004                                       5742      25  ...        172   
3006                                      11764      43  ...        433   
...                                         ...     ...  ...        ...   
3990                                          0       0  ...          5   
3991                                          0       0  ...          7   
3992                                          0       0  ...          6   
3995                                         26       4  ...         87   
3996                                         20       1  ...         54   

                              totalvehs                              owndwell  \
                                      0     1     2    3   4+ Being Purchased   
(zone_id, Unnamed: 0_level_1)                                                   
3000                              14706  5436   853  412  399            2452   
3002                                465  1271   556   89   66             435   
3003                               1396  1934   429   94   54             797   
3004                               1207  3448   867  166   97            1206   
3006                               4740  5860  1040  192  146            2132   
...                                 ...   ...   ...  ...  ...             ...   
3990                                 11    11    30   23   21              16   
3991                                 16    44    78   40   33              51   
3992                                 12   153   218   93   79             229   
3995                                370  1946  1704  582  342            1326   
3996                                 89  1049  1198  295  134             687   

                                                                       
                              Being Rented Fully Owned Something Else  
(zone_id, Unnamed: 0_level_1)                                          
3000                                 16594        2290            470  
3002                                  1292         678             42  
3003                                  2548         463             99  
3004                                  3210        1271             98  
3006                                  8101        1534            211  
...                                    ...         ...            ...  
3990                                     0          22             58  
3991                                    17          67             76  
3992                         

In [5]:
# Now based on the diff, decide which one to del
# We can search for all negative cells and then find possible combinations to del them from the current kept
# Convert the kept to pandas.Series for quick search and delete
# NOTE: still need to optimise the relationship searching
# For each zone, find possible combinations to delete
# Find which comb exist in the kept df and then delete the smaller number (including the number in kept)
# The rest we will delete one by one

In [6]:
def add_0_to_missing(df, ls_missing, axis):
    for missing in ls_missing:
        if axis == 1: # by row
            df.loc[missing] = 0
        elif axis == 0: # by col
            df[missing] = 0
    return df

# make sure they both have the same rows and cols, if not it means 0
missing_cols_ori = set(new_marg_hh.columns) - set(new_ori_census.columns)
missing_cols_kept = set(new_ori_census.columns) - set(new_marg_hh.columns)
missing_rows_ori = set(new_marg_hh.index) - set(new_ori_census.index)
missing_rows_kept = set(new_ori_census.index) - set(new_marg_hh.index)

new_marg_hh = add_0_to_missing(new_marg_hh, missing_cols_kept, 0)
new_marg_hh = add_0_to_missing(new_marg_hh, missing_rows_kept, 1)
new_ori_census = add_0_to_missing(new_ori_census, missing_cols_ori, 0)
new_ori_census = add_0_to_missing(new_ori_census, missing_rows_ori, 1)

In [7]:
yo_te = new_ori_census - new_marg_hh
# yo_te.columns = ['_'.join(col) for col in yo_te.columns.values]
yo_te.index.name = "POA"
yo_te

dwelltype                                                hhinc  \
     Flat or Apartment Missing Other Separate House Terrace/Townhouse 1-149   
POA                                                                           
3000               219      39    59              4                64     2   
3002                28       5   -48             52                46   -18   
3003                56      -3    -5             10                38    -8   
3004               172       9     8            -73                 6   -28   
3006               156      17   -83            -73               158   -41   
...                ...     ...   ...            ...               ...   ...   
3990               -11       0    -4             36                14    -1   
3991                 0       0    -3             25                 4    -2   
3992                 0       0     0             27                 0     0   
3995                25       0  -163             94               140     0   
3996                20      -1     3             58               -19    -8   

                                            ... hhsize        owndwell  \
     1000-1249 1250-1499 150-299 1500-1749  ...     8+ Being Purchased   
POA                                         ...                          
3000       -14        10    -128        13  ...     49            -685   
3002        53         3     -26         6  ...     43             -20   
3003       -54         6     -67         4  ...     51             128   
3004        32       -21    -111         7  ...     51             390   
3006      -108         4    -212         8  ...     50             382   
...        ...       ...     ...       ...  ...    ...             ...   
3990         3         5       0         2  ...     34              -2   
3991         0         0       1         2  ...     25              -8   
3992         0         0      -3         3  ...     22              67   
3995       -10        -7     -46         4  ...     29             -91   
3996         0         0     -46         3  ...     28             -96   

                                             totalvehs                       
     Being Rented Fully Owned Something Else         0     1     2    3  4+  
POA                                                                          
3000         3533       -2581            118        84  5436 -4562 -617  44  
3002           22          69             12        19    23    60  -27   8  
3003          433        -520             55        14    44   217 -147 -32  
3004         -152        -110             -6         0    60   215 -151  -2  
3006          955       -1187             25        39  -147   785 -453 -49  
...           ...         ...            ...       ...   ...   ...  ...  ..  
3990          -24           3             58         0    -3    21    5  12  
3991          -18         -23             75        -1     7     7    6   7  
3992          -39         -73             72       -44    44     7    5  15  
3995          338        -235             84      -117   121    61   11  20  
3996           39         103             15        89   -88    35   11  14  

[691 rows x 44 columns]

In [51]:
# Posisble to query multi-index from part of it, we can have all possible
# We then sum and see which one is smaller (if the kept smaller then delete all)
# To get the the number of delete (incase the smallest number is not the sum in kept)
# we get dist, then use the code to integrized to get the final

def convert_to_dict_ls(tup):
    di = {}
    for a, b in tup:
        di.setdefault(a, []).append((a,b))
    return di

def wrapper_final(kept_hh, diff_census, geo_lev):
    count_kept = kept_hh.drop(columns=["hhid"]).value_counts()
    diff_census = diff_census.head(5)
    for zone, r in diff_census.iterrows():
        print(f"DOING deleting to match cencus diff for {zone}")
        before_sum = count_kept.loc[count_kept.index.get_level_values(geo_lev) == zone].sum()
        sub_count_kept = count_kept.loc[count_kept.index.get_level_values(geo_lev) == zone]
        prev_indexs = sub_count_kept.index
        neg_cols = r[r < 0]
        # re check with neg val
        dict_neg_v = convert_to_dict_ls(neg_cols.index)
        for i in range(len(dict_neg_v)):
            raws_before_comb = combinations(dict_neg_v.values(), len(dict_neg_v) - i)
            for raw in raws_before_comb:
                if neg_cols.sum() == 0:
                    break
                ls_pos_neg_comb =list(product(*raw))
                for comb in ls_pos_neg_comb:
                    # loop through each neg combs all
                    condi_check = True
                    to_del_n = np.inf
                    # search for sub df with combs and also dfind the 
                    for att, state in comb:
                        condi_check &= sub_count_kept.index.get_level_values(att) == state
                        if att != geo_lev:
                            check_v = neg_cols.loc[(att, state)] * -1
                            if check_v < to_del_n: 
                                to_del_n = check_v
                    filtered_combs_from_kept = sub_count_kept.loc[condi_check]

                    if len(filtered_combs_from_kept) == 0 or to_del_n == 0:
                        continue

                    sum_val = filtered_combs_from_kept.sum()
                    if sum_val < to_del_n:
                        to_del_n = sum_val
                    
                    # we need to spread the del_n by the dist
                    temp_hold_combs = filtered_combs_from_kept / sum_val
                    temp_hold_combs = temp_hold_combs * to_del_n

                    # First del by just normal rounding
                    to_del_first = np.floor(temp_hold_combs)
                    filtered_combs_from_kept = filtered_combs_from_kept - to_del_first
                    remaining_to_del = to_del_n - to_del_first.sum()
                    # we will spread the remaing to del for the top
                    filtered_combs_from_kept.sort_values(ascending = False, inplace=True)
                    filtered_combs_from_kept.iloc[:int(remaining_to_del)] -= 1
                    # Make sure there are no neg
                    print(filtered_combs_from_kept)
                    assert not any(filtered_combs_from_kept < 0)

                    # Update the count kept
                    sub_count_kept.loc[filtered_combs_from_kept.index] = filtered_combs_from_kept
                    neg_cols.loc[list(comb)] += to_del_n
                    sub_count_kept = sub_count_kept[sub_count_kept > 0]
        zero_indexes = set(prev_indexs) - set(sub_count_kept.index)
        count_kept.loc[sub_count_kept.index] = sub_count_kept
        count_kept.loc[list(zero_indexes)] = 0
        assert neg_cols.sum() == 0
        after_sum = count_kept.loc[count_kept.index.get_level_values(geo_lev) == zone].sum()
        print(f"FNISHED deleting {before_sum - after_sum} HHs to match cencus diff for {zone}")
    return count_kept
m = wrapper_final(kept_hh, yo_te, geo_lev="POA")

DOING deleting to match cencus diff for 3000
dwelltype          owndwell         hhinc      totalvehs  hhsize  Spouse  Child  Others  Grandchild  POA 
Flat or Apartment  Being Purchased  1000-1249  2          2       1       0      0       0           3000    42.0
                                                          1       0       0      0       0           3000     7.0
                                                          3       1       1      0       0           3000     7.0
                                                          2       0       1      0       0           3000     7.0
                                                          3       0       2      0       0           3000     3.0
Name: count, dtype: float64
dwelltype          owndwell         hhinc    totalvehs  hhsize  Spouse  Child  Others  Grandchild  POA 
Flat or Apartment  Being Purchased  150-299  2          2       1       0      0       0           3000    0.0
                                    